In [5]:
import pathlib
import os
import re, time
import numpy as np
import pandas as pd
import scipy.ndimage as ndimage
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,Dropout,Flatten,BatchNormalization,Activation
from tensorflow.keras.applications import NASNetLarge,VGG19
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [6]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.58.202.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.58.202.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


In [7]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [8]:
project_id = 'starlit-summit-281117'

In [9]:
from google.colab import auth
auth.authenticate_user()

In [10]:
!gcloud config set project {project_id}

Updated property [core/project].


In [11]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


In [12]:
def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [13]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [14]:
def image_example(image_string):
  label = image_labels[image_string]
  image_string2 = open(image_string, 'rb').read()
  image_shape = tf.image.decode_jpeg(image_string2).shape
  feature = {
    'image_raw': _bytes_feature(image_string2),
      'score': _float_feature(image_labels[image_string])}

  return tf.train.Example(features=tf.train.Features(feature=feature))

In [15]:
EPOCHS = 12
BATCH_SIZE = 64
IMAGE_SIZE = [331, 331]
DATASET_SIZE = (2735+5081)
start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005 * 8
rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

gcs_pattern = 'gs://beholder/behold.tfrecords'
validation_split = 0.
split = len(gcs_pattern) - int(len(gcs_pattern) * validation_split)
TRAIN_STEPS =  DATASET_SIZE // BATCH_SIZE


train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)


In [16]:
def load_dataset(dataset):
    # read from TFRecords. For optimal performance, use "interleave(tf.data.TFRecordDataset, ...)"
    # to read from multiple TFRecord files at once and set the option experimental_deterministic = False
    # to allow order-altering optimizations.
    dataset = dataset.map(read_tfrecord)
    dataset = dataset.map(data_augment)
    return dataset

In [17]:
def random_rotate_image(image):
  image = ndimage.rotate(image, np.random.uniform(-30, 30), reshape=False)
  return image

In [18]:
raw_image_dataset = tf.data.TFRecordDataset(gcs_pattern)

def _extract_fn(tfrecord):
    # Extract features using the keys set during creation
    features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'score': tf.io.FixedLenFeature([], tf.float32)
}
    # Extract the data record
    sample = tf.io.parse_single_example(tfrecord, features)

    image = tf.image.decode_jpeg(sample['image'],3)  
    print(image) 
    print(type(image))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0, 2)
    image = tf.image.resize_with_pad(image, 331,331 , method='lanczos3') 
    image = tf.image.convert_image_dtype(image, tf.uint32, saturate=False) 
    image = tf.cast(image, tf.float32)/255.
    score = sample['score']
    return [image, score]  

parsed_image_dataset = raw_image_dataset.map(_extract_fn)
parsed_image_dataset

Tensor("DecodeJpeg:0", shape=(None, None, 3), dtype=uint8)
<class 'tensorflow.python.framework.ops.Tensor'>


<MapDataset shapes: ((331, 331, 3), ()), types: (tf.float32, tf.float32)>

In [19]:
full_dataset = parsed_image_dataset
full_dataset = full_dataset.shuffle(DATASET_SIZE)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)
train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

In [20]:
with strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  pretrained_model = NASNetLarge(
      weights=None, 
      include_top=False, 
      input_shape=(331, 331, 3))
  model = tf.keras.Sequential([
            pretrained_model,
            GlobalMaxPooling2D(),
            Dense(1000),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(250),
            BatchNormalization(),
            Activation('relu'),
            Dropout(0.5),
            Dense(1, activation='linear')])
  model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['mean_absolute_error','mean_absolute_percentage_error'])
print('Done.')

Done.


In [21]:
def lrfn(epoch):
    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        if epoch < rampup_epochs:
            lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        else:
            lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
        return lr
    return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)
checkpoint = ModelCheckpoint('model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='min')
progbar_callback = tf.keras.callbacks.ProgbarLogger(count_mode='steps')
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='Graph', histogram_freq=1,  
          write_graph=True, write_images=True)

In [ ]:
start_time = time.time()

history = model.fit(train_dataset, validation_data=val_dataset,
                    steps_per_epoch=TRAIN_STEPS, epochs=EPOCHS,
                    callbacks=[lr_callback, checkpoint])

final_accuracy = history.history["val_loss"][-5:]
print("FINAL ACCURACY MEAN-5: ", np.mean(val_loss))
print("TRAINING TIME: ", time.time() - start_time, " sec")


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/12


In [ ]:
print(history.history.keys())
display_training_curves(history.history['accuracy'][1:], history.history['val_accuracy'][1:], 'accuracy', 211)
display_training_curves(history.history['loss'][1:], history.history['val_loss'][1:], 'loss', 212)

In [ ]:
some_flowers, some_labels = dataset_to_numpy_util(validation_dataset, 160)
# randomize the input so that you can execute multiple times to change results
permutation = np.random.permutation(8*20)
some_flowers, some_labels = (some_flowers[permutation], some_labels[permutation])

predictions = model.predict(some_flowers, batch_size=16)
evaluations = model.evaluate(some_flowers, some_labels, batch_size=16)
  
print(np.array(CLASSES)[np.argmax(predictions, axis=-1)].tolist())
print('[val_loss, val_acc]', evaluations)

display_9_images_with_predictions(some_flowers, predictions, some_labels)

In [ ]:
model.save('model.h5')